In [1]:
import pandas as pd
import numpy as np
from nltk import RegexpTokenizer
from nltk.corpus import stopwords

import nltk
nltk.download('stopwords')

import gensim

from sklearn.cluster import KMeans
from sklearn import metrics
import pylab as pl

import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

import warnings
warnings.filterwarnings('ignore')

import emoji
import regex as re
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.metrics import pairwise_distances

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/natalie/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Data 

In [2]:
data = pd.read_csv('scrape/restaurant-data/cleaned_restaurant_reviews.csv', index_col=0).reset_index(drop=True)
data.head(2)

,url,name,neighbourhood,price,categories,review,user,date,cleaned_price,cleaned_categories,...,Western,Recommended,Sustainable,Novel,Desserts,Bites,Supper,Breakfast & Brunch,Halal,region
0,https://www.burpple.com/bedok-85-market?bp_ref...,85 Fengshan Centre,Bedok,~$5/pax,"['Hawker Food', 'Supper', 'Cheap & Good']","\nFish Ball Minced Meat Noodle\nFishball, meat...",Triffany Lim,21m ago,5.0,"['Local Delights', 'Supper', 'Value']",...,0,0,0,0,0,0,1,0,0,East
1,https://www.burpple.com/bedok-85-market?bp_ref...,85 Fengshan Centre,Bedok,~$5/pax,"['Hawker Food', 'Supper', 'Cheap & Good']","\nOrh lua\nThere are a couple of stores, but g...",Ally Tan,Jul 30 at 4:12pm,5.0,"['Local Delights', 'Supper', 'Value']",...,0,0,0,0,0,0,1,0,0,East


### Review Text Cleaning
(to shift to eda code + run with translation)

In [3]:
ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [4]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"didn\'t", "did not", phrase)
    phrase = re.sub(r"don\'t", "do not", phrase)
    phrase = re.sub(r"o\'clock", "clock", phrase)
    phrase = re.sub(r"couldn\'t", "could not", phrase)
    phrase = re.sub(r"that\'s", "that is", phrase)       
    phrase = re.sub(r"go-around", "go around", phrase)  
    # general
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    #phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [5]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

In [6]:
stopwords = nltk.corpus.stopwords.words('english')
new_stopwords = ['address', 'note', 'tel', 'website', 'open', 'burpple']
add_stopwords = set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
    "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
    'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
    'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
    'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
    'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
    'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
    'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
    'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
    'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
    's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
    've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
    "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
    "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
    'won', "won't", 'wouldn', "wouldn't"])
add_stopwords_2 =  ['n','s','m','i','1','2','3','4','5','6','7','8','9','10','one','two',
    'it','in','ve','well','could','would','really','also','even',
    'alway','always','still','never','much','thing','yet',
    'said','asked','did','go','got','do','make','know','think','come','going',
    'put','went','seem','order','ordered','give','eat','make','get']

stopwords.extend(new_stopwords)
stopwords.extend(add_stopwords)
stopwords.extend(add_stopwords_2)

In [7]:
cleaned_review_list = []
count = 0

for review in data['review']:
    # lower case
    review = review.lower()

    # deemojize
    review = emoji.demojize(review)

    # remove headers
    review = ' '.join(review.split('\n')[2:])

    # remove location (pushpin or location:)
    review = review.split('round_pushpin')[0]
    review = review.split('location:')[0]

    # clean punctuation
    review = re.sub(r'[^\w\s]', '', review)

    # tokenize
    tokens = word_tokenize(review)

    # remove stopwords and URLs
    tokens = [word for word in tokens if word not in stopwords]
    tokens = [word for word in tokens if 'http' not in word]
    tokens = [word for word in tokens if 'www' not in word]

    # decontraction
    tokens = [decontracted(word) for word in tokens]

    ### to add in translation code... shld translate each token

    ### lemmatization
    # POS tagging
    tokens = [nltk.pos_tag([word]) for word in tokens]
            
    # lemmatization
    tokens = [lemmatizer.lemmatize(word[0][0], get_wordnet_pos(word[0][1])) 
        if get_wordnet_pos(word[0][1])!=None else lemmatizer.lemmatize(word[0][0]) for word in tokens]
    
    ### WORD FREQUENCY
    # modify count if POS tag is adjective
    # new_tokens = []
    # for token in tokens:
    #     new_tokens.append(token)
    #     word, pos = nltk.pos_tag([token])[0]
    #     if pos == 'JJ' or pos == 'JJR' or pos == 'JJS' or pos == 'RB' or pos == 'RBR' or pos == 'RBS':  
    #         new_tokens.append(token) ### ADD 1 MORE WORD OCCURENCE
        #     if word in word_freq.keys():
        #         word_freq[word] += 2 ### TO CHANGE THIS PARAMETER (double frequency for adjectivies)
        #     else: 
        #         word_freq[word] = 2
        # else:
        #     if word in word_freq.keys():
        #         word_freq[word] += 1 
        #     else: 
        #         word_freq[word] = 1
        
    # concatenate tokens back
    cleaned_review = " ".join(tokens)
    cleaned_review_list.append(cleaned_review)

    if count%1000 == 0:
        print(count)
    count+=1

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [ ]:
# replace the old cleaned text
data['cleaned_text'] = cleaned_review_list

In [ ]:
# concatenate all the reviews for each restaurant
restaurant_review_df = data[['url', 'cleaned_text']]
restaurant_review_df = restaurant_review_df.groupby(['url'], as_index = False).agg({'cleaned_text': ' '.join})
restaurant_review_df.head(5)

In [ ]:
restaurant_characteristics_df = data.drop(['review', 'cleaned_text'], axis=1)
restaurant_characteristics_df = restaurant_characteristics_df.groupby(['url'], as_index = False).first()
restaurant_characteristics_df.head(2)

In [ ]:
restaurant_review_df = pd.merge(restaurant_review_df, restaurant_characteristics_df, on='url')

# Model

In [ ]:
tokenizer = RegexpTokenizer(r'\w+')

def nlp_clean(data):
    new_data = []
    for d in data:
        new_str = d.lower()
        dlist = tokenizer.tokenize(new_str)
        new_data.append(dlist)
    return new_data

class LabeledLineSentence(object):
    def __init__(self, doc_list, labels_list):
        self.labels_list = labels_list
        self.doc_list = doc_list
    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
              yield gensim.models.doc2vec.TaggedDocument(doc, [self.labels_list[idx]])

In [ ]:
# tokenize reviews
tokenized_reviews = nlp_clean(restaurant_review_df['cleaned_text'])

## Functions

In [ ]:
# doc2vec
# word embeddings: calculate use frequency -> calculate cosine similarity -> group based on similar usage of words. (LDA just the first step)
# vector size 1000 == randomly selecting 1000

def doc2vec(tokenized_reviews, num_epochs): #, word_freq
    it = LabeledLineSentence(tokenized_reviews, restaurant_review_df['url']) # label reviews with the restaurant url
    model = gensim.models.Doc2Vec(vector_size=1000, min_count=5, alpha=0.025, min_alpha=0.025, seed=123) #### TO TUNE
    # model.build_vocab_from_freq(word_freq) #### TRY BUILD VOCAB FROM FREQ
    model.build_vocab(it)
    model.train(it, total_examples= model.corpus_count, epochs = num_epochs, start_alpha=0.002, end_alpha=-0.016)

    # print(model.corpus_count) ## check the corpus
    # print(len(model.docvecs)) # check doc2vec.

    ### plot the accuracy of each epoch so we can see if its over or under-fitted
    ### tweak learning rate and number of epochs
    return it, model

In [ ]:
def optimise_epochs(tokenized_reviews, max_epoch):
    range_epochs = range(2,max_epoch+1)
    accuracy = []
    for num_epoch in range_epochs:
        it, model = doc2vec(tokenized_reviews, num_epoch)

        kmeans = KMeans(n_clusters=5, init='k-means++', max_iter=100) ### ASSUME 5 clusters
        X = kmeans.fit(model.dv.vectors)
        silhouette_score_average = silhouette_score(model.dv.vectors, kmeans.predict(model.dv.vectors))
        accuracy.append(silhouette_score_average)

    plt.plot(list(range_epochs),accuracy)
    plt.title('Silhouette Score For Each Epoch')
    plt.show()

In [ ]:
def plot_silhouette(tokenized_reviews):
    it, model = doc2vec(tokenized_reviews, 10)

    kmeans = KMeans(n_clusters=5, init='k-means++', max_iter=100) ### ASSUME 5 clusters
    X = kmeans.fit(model.dv.vectors)
    silhouette_score_average = silhouette_score(model.dv.vectors, kmeans.predict(model.dv.vectors))
    return silhouette_score_average

In [ ]:
def cosine_similarity_matrix(model):
    similarity_df = pd.DataFrame()

    for restaurant in it.labels_list:
        similarity_list = []
        for restaurant2 in it.labels_list:
            similarity_list.append(model.docvecs.similarity(restaurant, restaurant2))
        
        similarity_df[restaurant] = similarity_list

    similarity_df.index = list(it.labels_list)

    return similarity_df

In [ ]:
def num_topics_elbow(model):
    # get optimal number of topics
    Sum_of_squared_distances = []
    K = range(1,15)
    for k in K:
        kmeans = KMeans(n_clusters=k, init='k-means++', max_iter=100)
        X = kmeans.fit(model.dv.vectors) ###using documnet vecter numbers. doctag_syn0 calls doc2vec that is trained earlier.
        Sum_of_squared_distances.append(kmeans.inertia_)

    plt.plot(K, Sum_of_squared_distances, 'bx-')
    plt.xlabel('k')
    plt.ylabel('Sum_of_squared_distances')
    plt.title('Elbow Method For Optimal k')
    plt.show()

def num_topics_silhouette(model, min_topic, max_topic): # min topic should be the first elbow observed
    ### K-means Accuracy (Silhouette Score) (assume minimum of 5 clusters)
    accuracy = []
    for k in range(min_topic,max_topic+1):
        kmeans = KMeans(n_clusters=k, init='k-means++', max_iter=100)
        X = kmeans.fit(model.dv.vectors) ###using documnet vecter numbers. doctag_syn0 calls doc2vec that is trained earlier.
        silhouette_score_average = silhouette_score(model.dv.vectors, kmeans.predict(model.dv.vectors))
        accuracy.append(silhouette_score_average)
    x = list(range(min_topic,max_topic+1))
    print(accuracy)
    plt.plot(x,accuracy)
    plt.title('Silhouette Score per K')
    plt.show()


In [ ]:
def pca_plot(model):
    l = kmeans.fit_predict(model.dv.vectors)
    pca = PCA(n_components=2).fit(model.dv.vectors)
    datapoint = pca.transform(model.dv.vectors)

    plt.figure
    label1 = ["#FF6961", "#FFB480", "#F8F38D", "#42D6A4", "#08CAD1", "#59ADF6", "#9D94FF", "#C780E8"]
    #label1 = ["#FF355E", "#FD5B78", "#FF6037", "#FF9966", "#FF9933", "#FFCC33", "#FFFF66", "#CCFF00", "#66FF66", "#AAF0D1", "#16D0CB", "#50BFE6", "#9C27B0", "#EE34D2", "#FF00CC"]
    color = [label1[i] for i in labels]
    plt.scatter(datapoint[:, 0], datapoint[:, 1], c=color)

    centroids = kmeans.cluster_centers_
    centroidpoint = pca.transform(centroids)
    plt.scatter(centroidpoint[:, 0], centroidpoint[:, 1], marker='^', s=150, c='#000000')
    plt.show()

In [ ]:
def nearest_restaurants_to_centroid(model):
    # label each restaurant under 1 of the topics
    restaurant_review_df['topics'] = labels

    # count of restaurants under each topic
    print(restaurant_review_df['topics'].value_counts())

    distances = pairwise_distances(kmeans.cluster_centers_, model.dv.vectors, metric='euclidean')
    closest_indexes = [np.argpartition(i, 10)[:10] for i in distances] # for each centroid, get the 10 nearest restaurants

    interpret_df = pd.DataFrame(columns=['topics', 'url', 'name', 'cleaned_text'])
    for idx in closest_indexes:
        interpret_df = interpret_df.append(restaurant_review_df[['url', 'name', 'cleaned_text', 'topics']].iloc[idx])

    interpret_df = interpret_df.sort_values(['topics'])
    return restaurant_review_df, interpret_df

## Model 1
Base model

#### Train Model

In [ ]:
# identify optimal number of epochs
# plot k means silhouette score for each doc2vec epoch
optimise_epochs(tokenized_reviews, 10)

In [ ]:
it, model1 = doc2vec(tokenized_reviews, 3) ### Change number of epochs accordingly

### Similarity Matrix

In [ ]:
cosine_similarity_matrix(model1)

### Text Clustering Using K-Means

In [ ]:
num_topics_elbow(model1)

In [ ]:
num_topics_silhouette(model1, 3, 8) ### CHANGE ACCORDINGLY

#### Cluster Visualisation

In [ ]:
num_clusters = 4 ### CHANGE ACCORDINGLY 
kmeans = KMeans(n_clusters=num_clusters, init='k-means++', max_iter=100) 
X = kmeans.fit(model1.dv.vectors) ### CHANGE MODEL NUMBER
labels = kmeans.labels_.tolist()

In [ ]:
pca_plot(model1)

#### Cluster Interpretation

In [ ]:
restaurant_review_df, interpret_df = nearest_restaurants_to_centroid(model1)

In [ ]:
interpret_df[interpret_df['topics'] == 0]

In [ ]:
restaurant_review_df_encoded = restaurant_review_df.drop(['name', 'cleaned_text', 'price', 'categories', 'review', 'user', 'date', 'cleaned_categories'], axis=1)

# one hot encode region & topics
region_dummies = pd.get_dummies(restaurant_review_df_encoded['region'])
neighbourhood_dummies = pd.get_dummies(restaurant_review_df_encoded['neighbourhood'])
topic_dummies = pd.get_dummies(restaurant_review_df_encoded['topics'])

restaurant_review_df_encoded = restaurant_review_df_encoded.drop(['region', 'neighbourhood', 'topics'], axis=1)
restaurant_review_df_encoded = restaurant_review_df_encoded.join(region_dummies)
restaurant_review_df_encoded = restaurant_review_df_encoded.join(neighbourhood_dummies)
restaurant_review_df_encoded = restaurant_review_df_encoded.join(topic_dummies)

restaurant_review_df_encoded.to_csv('doc2vec_labelled_restaurants_model1.csv') ### CHANGE MODEL NAME

## Model 2
Appended additional adjective words

In [ ]:
def create_adj_tokenized_reviews(n):
    adj_tokenized_reviews = []
    for review in tokenized_reviews:
        adj_tokenized = []
        for token in review:
            adj_tokenized.append(token)
            word, pos = nltk.pos_tag([token])[0]
            if pos == 'JJ' or pos == 'JJR' or pos == 'JJS' or pos == 'RB' or pos == 'RBR' or pos == 'RBS':
                for i in range(n):
                    adj_tokenized.append(token)
                
        adj_tokenized_reviews.append(adj_tokenized)
    return adj_tokenized_reviews

In [ ]:
adj_tokenized_reviews = create_adj_tokenized_reviews(1)

In [ ]:
len(adj_tokenized_reviews[0])

In [ ]:
len(tokenized_reviews[0])

#### Train Model

In [ ]:
# identify optimal weights. Assume 5 clusters, 10 epochs 
scores = []
range_weights = range(1,4)
for i in range_weights:
    adj_tokenized_reviews = create_adj_tokenized_reviews(i)
    scores.append(plot_silhouette(adj_tokenized_reviews))

plt.plot(list(range_weights),scores)
plt.title('Silhouette Score For Each Added Frequencies')
plt.show()

In [ ]:
# identify optimal number of epochs
adj_tokenized_reviews = create_adj_tokenized_reviews(2)
optimise_epochs(adj_tokenized_reviews, 10)

In [ ]:
it, model2 = doc2vec(adj_tokenized_reviews, 4) ### Change number of epochs accordingly

### Similarity Matrix

In [ ]:
cosine_similarity_matrix(model2)

### Text Clustering Using K-Means

In [ ]:
num_topics_elbow(model2)

In [ ]:
num_topics_silhouette(model2, 3, 8) ### CHANGE ACCORDINGLY

#### Cluster Visualisation

In [ ]:
num_clusters = 3 ### CHANGE ACCORDINGLY
kmeans = KMeans(n_clusters=num_clusters, init='k-means++', max_iter=100) 
X = kmeans.fit(model2.dv.vectors) ### CHANGE MODEL NUMBER
labels = kmeans.labels_.tolist()

In [ ]:
pca_plot(model2)

#### Cluster Interpretation

In [ ]:
restaurant_review_df, interpret_df = nearest_restaurants_to_centroid(model2)

In [ ]:
interpret_df[interpret_df['topics'] == 0]

In [ ]:
restaurant_review_df_encoded = restaurant_review_df.drop(['name', 'cleaned_text', 'price', 'categories', 'review', 'user', 'date', 'cleaned_categories'], axis=1)

# one hot encode region & topics
region_dummies = pd.get_dummies(restaurant_review_df_encoded['region'])
neighbourhood_dummies = pd.get_dummies(restaurant_review_df_encoded['neighbourhood'])
topic_dummies = pd.get_dummies(restaurant_review_df_encoded['topics'])

restaurant_review_df_encoded = restaurant_review_df_encoded.drop(['region', 'neighbourhood', 'topics'], axis=1)
restaurant_review_df_encoded = restaurant_review_df_encoded.join(region_dummies)
restaurant_review_df_encoded = restaurant_review_df_encoded.join(neighbourhood_dummies)
restaurant_review_df_encoded = restaurant_review_df_encoded.join(topic_dummies)

restaurant_review_df_encoded.to_csv('doc2vec_labelled_restaurants_model2.csv') ### CHANGE MODEL NAME

## Model 3
Bi-gram

In [ ]:
from gensim.models.phrases import Phrases, Phraser
# Build the bigram and trigram models
bigram = gensim.models.Phrases(tokenized_reviews) # higher threshold fewer phrases.

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

bigram_reviews = make_bigrams(tokenized_reviews)

#### Train Model

In [ ]:
# identify optimal number of epochs
# plot k means silhouette score for each doc2vec epoch
optimise_epochs(bigram_reviews, 10)

In [ ]:
it, model3 = doc2vec(bigram_reviews, 6) ### Change number of epochs accordingly

### Similarity Matrix

In [ ]:
cosine_similarity_matrix(model3)

### Text Clustering Using K-Means

In [ ]:
num_topics_elbow(model3)

In [ ]:
num_topics_silhouette(model3, 3, 8) ### CHANGE ACCORDINGLY

#### Cluster Visualisation

In [ ]:
num_clusters = 3 ### CHANGE ACCORDINGLY 
kmeans = KMeans(n_clusters=num_clusters, init='k-means++', max_iter=100) 
X = kmeans.fit(model3.dv.vectors) ### CHANGE MODEL NUMBER
labels = kmeans.labels_.tolist()

In [ ]:
pca_plot(model3)

#### Cluster Interpretation

In [ ]:
restaurant_review_df, interpret_df = nearest_restaurants_to_centroid(model3)

In [ ]:
interpret_df[interpret_df['topics'] == 0]

In [ ]:
restaurant_review_df_encoded = restaurant_review_df.drop(['name', 'cleaned_text', 'price', 'categories', 'review', 'user', 'date', 'cleaned_categories'], axis=1)

# one hot encode region & topics
region_dummies = pd.get_dummies(restaurant_review_df_encoded['region'])
neighbourhood_dummies = pd.get_dummies(restaurant_review_df_encoded['neighbourhood'])
topic_dummies = pd.get_dummies(restaurant_review_df_encoded['topics'])

restaurant_review_df_encoded = restaurant_review_df_encoded.drop(['region', 'neighbourhood', 'topics'], axis=1)
restaurant_review_df_encoded = restaurant_review_df_encoded.join(region_dummies)
restaurant_review_df_encoded = restaurant_review_df_encoded.join(neighbourhood_dummies)
restaurant_review_df_encoded = restaurant_review_df_encoded.join(topic_dummies)

restaurant_review_df_encoded.to_csv('doc2vec_labelled_restaurants_model3.csv') ### CHANGE MODEL NAME

## Model 4
Tri-gram

In [ ]:
from gensim.models.phrases import Phrases, Phraser
# Build the bigram and trigram models
bigram = gensim.models.Phrases(tokenized_reviews) # higher threshold fewer phrases.

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)

trigram = gensim.models.Phrases(bigram_mod['tokenized_reviews'])
trigram_mod = gensim.models.phrases.Phraser(trigram)


def make_trigrams(texts):
    return [trigram_mod[doc] for doc in texts]

trigram_reviews = make_trigrams(tokenized_reviews)

#### Train Model

In [ ]:
# identify optimal number of epochs
# plot k means silhouette score for each doc2vec epoch
optimise_epochs(trigram_reviews, 10)

In [ ]:
it, model4 = doc2vec(trigram_reviews, 3) ### Change number of epochs accordingly

### Similarity Matrix

In [ ]:
cosine_similarity_matrix(model4)

### Text Clustering Using K-Means

In [ ]:
num_topics_elbow(model4)

In [ ]:
num_topics_silhouette(model4, 3, 8) ### CHANGE ACCORDINGLY

#### Cluster Visualisation

In [ ]:
num_clusters = 4 ### CHANGE ACCORDINGLY 
kmeans = KMeans(n_clusters=num_clusters, init='k-means++', max_iter=100) 
X = kmeans.fit(model4.dv.vectors) ### CHANGE MODEL NUMBER
labels = kmeans.labels_.tolist()

In [ ]:
pca_plot(model4)

#### Cluster Interpretation

In [ ]:
restaurant_review_df, interpret_df = nearest_restaurants_to_centroid(model4)

In [ ]:
interpret_df[interpret_df['topics'] == 0]

In [ ]:
restaurant_review_df_encoded = restaurant_review_df.drop(['name', 'cleaned_text', 'price', 'categories', 'review', 'user', 'date', 'cleaned_categories'], axis=1)

# one hot encode region & topics
region_dummies = pd.get_dummies(restaurant_review_df_encoded['region'])
neighbourhood_dummies = pd.get_dummies(restaurant_review_df_encoded['neighbourhood'])
topic_dummies = pd.get_dummies(restaurant_review_df_encoded['topics'])

restaurant_review_df_encoded = restaurant_review_df_encoded.drop(['region', 'neighbourhood', 'topics'], axis=1)
restaurant_review_df_encoded = restaurant_review_df_encoded.join(region_dummies)
restaurant_review_df_encoded = restaurant_review_df_encoded.join(neighbourhood_dummies)
restaurant_review_df_encoded = restaurant_review_df_encoded.join(topic_dummies)

restaurant_review_df_encoded.to_csv('doc2vec_labelled_restaurants_model4.csv') ### CHANGE MODEL NAME

## (IGNORE) WordCloud

In [ ]:
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# concatenate all the reviews for each topic
topic_df = interpret_df[['topics', 'cleaned_text']]
topic_df = topic_df.groupby(['topics'], as_index = False).agg({'cleaned_text': ' '.join})
topic_df

In [ ]:
# TF-IDF
text = topic_df['cleaned_text']
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(text)
feature_names = vectorizer.get_feature_names_out()
dense = X.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names).transpose()
df.columns = topic_df['topics']

In [ ]:
wordcloud = WordCloud(background_color = 'white', collocations = False).generate_from_text(text[0])
# Display the generated Word Cloud
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

In [ ]:
wordcloud = WordCloud(background_color = 'white', collocations = False).generate_from_frequencies(df[0])
# Display the generated Word Cloud
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

In [ ]:
wordcloud = WordCloud(background_color = 'white', collocations = False).generate_from_text(text[1])
# Display the generated Word Cloud
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

In [ ]:
wordcloud = WordCloud(background_color = 'white', collocations = False).generate_from_frequencies(df[1])
# Display the generated Word Cloud
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

In [ ]:
wordcloud = WordCloud(background_color = 'white', collocations = False).generate_from_frequencies(df[2])
# Display the generated Word Cloud
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

In [ ]:
wordcloud = WordCloud(background_color = 'white', collocations = False).generate_from_frequencies(df[3])
# Display the generated Word Cloud
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

In [ ]:
wordcloud = WordCloud(background_color = 'white', collocations = False).generate_from_frequencies(df[4])
# Display the generated Word Cloud
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

In [ ]:
wordcloud = WordCloud(background_color = 'white', collocations = False).generate_from_frequencies(df[5])
# Display the generated Word Cloud
plt.imshow(wordcloud)
plt.axis("off")
plt.show()